### Занятие. Наибольшая возрастающая подпоследовательность

In [42]:
def largestIncreasingSubsequence(A, n):
    D = [0] * n
    prev = [0] * n
    for i in range(n):
        D[i] = 1
        prev[i] = -1
        for j in range(i):
            if A[i] > A[j] and D[j] + 1 > D[i]:
                D[i] = D[j] + 1
                prev[i] = j
                
    k = max(D)
    ind = D.index(max(D))  # индекс максимума
    lis_ind = [0] * k
    for i in range(k):
        lis_ind[k-i-1] = ind
        ind = prev[ind]
    return k, lis_ind

In [24]:
a = [7, 2, 1, 3, 8, 4, 9, 1, 2, 6, 5, 9, 3, 8, 1]
k, inds = largestIncreasingSubsequence(a, len(a))
k, inds

(5, [1, 3, 5, 9, 11])

### Задание 1. Наибольшая последовательнократная подпоследовательность

Дано целое число $1≤n≤10^3$ и массив $A[1…n]$ натуральных чисел, не превосходящих $2⋅10^9$. Выведите максимальное $1≤k≤n$, для которого найдётся подпоследовательность $1≤i_1<i_2<…<i_k≤n$ длины $k$, в которой каждый элемент делится на предыдущий (формально: для  всех $1≤j<k$, $A[i_{j}] ∣ A[i_{j+1}]$).

In [32]:
def largestDivisibleSubsequence(A, n):
    D = [0] * n
    for i in range(n):
        D[i] = 1
        for j in range(i):
            if (A[i] % A[j] == 0) and (D[j] + 1 > D[i]):
                D[i] = D[j] + 1
    return max(D)

In [34]:
a = [3, 6, 7, 12]
k = largestDivisibleSubsequence(a, len(a))
k

3

### Задание 2. Наибольшая невозрастающая подпоследовательность
Сложность: $nlogn$

Дано целое число $1≤n≤10^5$ и массив $A[1…n]$, содержащий неотрицательные целые числа, не превосходящие 10^9. Найдите наибольшую невозрастающую подпоследовательность в $A$. В первой строке выведите её длину $k$, во второй — её индексы $1≤i_1<i_2<…<i_k≤n$ (таким образом, $A[i_1]≥A[i_2]≥…≥A[i_n]$).

In [223]:
# https://foxford.ru/wiki/informatika/naibolshaya-vozrastayuschaya-podposledovatelnost
def LDS_nlogn(A, n):
    INF = 10 ** 10
    F = [-INF] * (n + 1)  # F монотонно убывает
    F[0] = INF
    prev = []
    for i in range(n):
        l, r = 0, n
        while r > l + 1:
            middle = (l + r) // 2
            if F[middle] < A[i]:
                r = middle
            else:
                l = middle
        F[r] = A[i]
        prev.append([r, i, A[i]])
    ans = 0
    for i in range(n+1):
        if F[i] != INF and F[i] != -INF:
            ans += 1
            
    index_list = [0] * ans
    cnt = ans

    for k, i, _ in prev[::-1]:
        if k == cnt:
            index_list[k-1] = i+1
            cnt -= 1
            
    return ans, index_list

seq = [5, 3, 4, 4, 2, 5, 9]
k, inds = LDS_nlogn(seq, len(seq))
print(k)
print((" ").join(map(str, inds)))

4
1 3 4 5


In [224]:
a = [1]
b = [2, 1]
c = [5, 3, 4, 4, 2, 5, 9]
d = [i for i in range(100, 0, -1)]

assert LDS_nlogn(a, len(a)) == (1, [1]), "WA"
assert LDS_nlogn(b, len(b)) == (2, [1, 2]), "WA"
assert LDS_nlogn(c, len(c)) == (4, [1, 3, 4, 5]), "WA"
assert LDS_nlogn(d, len(d)) == (100, list(range(1, 101))), "WA"

### Задание 3. Расстояние редактирования (Левенштейна)

Вычислите расстояние редактирования двух данных непустых строк длины не более $10^2$, содержащих строчные буквы латинского алфавита.

In [207]:
def edit_distance(s1, s2):
    try:
        n, m = len(s1), len(s2)
    except:
        print("Input must be str()")
    else:
        D = [[0] * (m+1) for _ in range(n+1)]

        for i in range(n + 1):
            D[i][0] = i
        for j in range(m + 1):
            D[0][j] = j

        for i, ch1 in enumerate(s1, start=1):
            for j, ch2 in enumerate(s2, start=1):
                D[i][j] = min(D[i-1][j] + 1,
                              D[i][j-1] + 1,
                              D[i-1][j-1] + (ch1 != ch2))
        return D[n][m]

# s1, s2 = input(), input()
# print(edit_distance(s1, s2))

In [212]:
edit_distance(1, "ab")

Input must be str()


In [214]:
assert edit_distance("ab", "ab") == 0, "WA"
assert edit_distance("short", "ports") == 3, "WA"

### Занятие. Рюкзак с повторениями

Создаем массив стоимости рюкзака $D[0...0]$ и заполним снизу вверх Bottom Up\
Оптимальность: $D[w]$ = max стоимость рюкзака вместимости w\
Подзадача: заполнить рюкзак вместимости $w - w_i$

$$D[w] = \max_{w_i \le w} D[w - w_i] + c_i$$

In [152]:
W, n = 10, 5
w_list = [3, 5, 6, 1, 9]
c_list = [400, 200, 700, 120, 1000]

def KnapSackBU_Reps(W, n, w_list, c_list):
    D = [0] * (W+1)
    
    for w in range(1, W+1):
        for i in range(n):
            w_i, c_i = w_list[i], c_list[i]
            
            if w_i <= w:
                D[w] = max(D[w], D[w - w_i] + c_i)

    return D[W]

In [153]:
KnapSackBU_Reps(W, n, w_list, c_list)
# 400 + 400 + 400 + 120

1320

### Задача 4. Рюкзак без повторений

Первая строка входа содержит целые числа $1≤W≤10^4$ и $1≤n≤300$ — вместимость рюкзака и число золотых слитков. Следующая строка содержит $n$ целых чисел $0≤w_1,…,w_n≤10^5$, задающих веса слитков. Найдите максимальный вес золота, который можно унести в рюкзаке.

**Решение:** \
$w_i$ - вес слитка $i$ \
$c_i = w_i$ - стоимость слитка $i$ пропорциональна весу, максимизируем вес слитков \
$D[w, i]$ = max стоимость рюкзака вместимости $w$, при использовании предметов $1...i$ \
!Важно предметы отсортированы по весу

Предмет $i$ либо используем, либо не используем
$$
D[w, i] = \max_{w_i \le w} D[w - w_i, i - 1] + c_i, \\
D[w, i] = D[w, i - 1]
$$
Выбираем максимум из двух вариантов

In [11]:
W, n = 10, 3
w_arr = [1, 4, 8]

In [28]:
def KnapSackBU_NoReps(W, n, w_arr):
    D = [[0] * (n+1) for _ in range(W+1)]

    for i in range(1, n+1):
        w_i = w_arr[i-1]

        for w in range(1, W+1):

            D[w][i] = D[w][i-1]  # не используя предмет i
            if w_i <= w:         # обновляем, если лучше использовать предмет i
                D[w][i] = max(D[w-w_i][i-1] + w_i, D[w][i]) 

    return D[W][n]

In [ ]:
W, n = map(int, input().split())
w_arr = list(map(int, input().split()))

print(KnapSackBU_NoReps(W, n, w_arr))

### Задача 5. Лестница

Даны число $1≤n≤10^2$ ступенек лестницы и целые числа $−10^4≤a_1,…,a_n≤10^4$, которыми помечены ступеньки. Найдите максимальную сумму, которую можно получить, идя по лестнице снизу вверх (от нулевой до $n$-й ступеньки), каждый раз поднимаясь на одну или две ступеньки.

**Решение:** \
Оптимальность: если ступень $i$ оптимальна (дает макс сумму), то либо ступень $i-1$, либо $i-2$ тоже оптимальна \
$D[i]$ - максимальная сумма ступенек на ступеньке $i$ \
Подзадачи: $D[i-1]$, $D[i-2]$

In [124]:
def StairsBU(n, slabs):
    if n == 1:
        return slabs[0]
    
    INF = 10 ** 5
    D = [-INF] * (n+1)
    D[0] = slabs[0]
    D[1] = max(slabs[0]+slabs[1], slabs[1])
    
    for i in range(2, n):
        s_i = slabs[i]
        D[i] = max(D[i-1] + s_i, D[i-2] + s_i)
        
    return D[n-1]

In [125]:
n = 3
slabs = [2, -1, 1]
StairsBU(n, slabs)

3

### Задача 6. Примитивный калькулятор

У вас есть примитивный калькулятор, который умеет выполнять всего три операции с текущим числом $x$: заменить $x$ на $2x$, $3x$ или $x+1$. По данному целому числу $1≤n≤10^5$ определите минимальное число операций $k$, необходимое, чтобы получить $n$ из $1$. Выведите $k$ и последовательность промежуточных чисел.

**Решение:** \
Надо получить число n методом Bottom Up, считаем число 1 оптимальным \
$D[i]$ - количество операций чтобы получить число i \
Подзадачи: $D[i / 3]$, $D[i / 2]$ (если делятся), $D[i - 1]$

D[1] = 0 \
D[2] = D[1] + 1 or D[2 / 2] + 1 = 1 \
D[3] = D[2] + 1 or D[3 / 3] + 1 = 1 \
D[4] = D[3] + 1 or D[4 / 2] + 1 = 2 \
D[5] = D[4] + 1 = 3 \
D[6] = D[5] + 1 or D[3] + 1 or D[2] + 1 = 4 or 2 or 2 = 2 \
...

In [78]:
def CalcBU(n):
    D = [0] * (n+1)
    
    for i in range(2, n+1):
        
        if (i % 2 == 0) and (i % 3 == 0):
            D[i] = min(D[i-1] + 1, D[i//2] + 1, D[i//3] + 1)
        elif i % 2 == 0:
            D[i] = min(D[i-1] + 1, D[i//2] + 1)
        elif i % 3 == 0:
            D[i] = min(D[i-1] + 1, D[i//3] + 1)
        else:
            D[i] = D[i-1] + 1
                    
    k = D[n]
    oper = [n]
    for _ in range(k):
        
        if (n % 3 == 0) and (D[n//3] == D[n] - 1):
            n //= 3
        elif (n % 2 == 0) and (D[n//2] == D[n] - 1):
            n //= 2
        else:
            n -= 1
            
        oper.append(n)
        
    return k, oper[::-1]

In [79]:
k, oper = CalcBU(96234)
k, oper

(14,
 [1, 3, 9, 10, 11, 22, 66, 198, 594, 1782, 5346, 16038, 16039, 32078, 96234])

In [80]:
' '.join(map(str, oper))

'1 3 9 10 11 22 66 198 594 1782 5346 16038 16039 32078 96234'